https://github.com/pyg-team/pytorch_geometric/blob/master/examples/graph_sage_unsup.py 
on CORA dataset

In [ ]:
train_loader = NeighborLoader(data, 
                            input_nodes=data.train_mask,
                            num_neighbors=[neighborhood_1, neighborhood_2], 
                            shuffle=True)

# TODO: not sure if subgraph is needed
subgraph_loader = NeighborLoader(copy.copy(data), input_nodes=None,
                                num_neighbors=[-1], shuffle=False)
# No need to maintain these features during evaluation:
del subgraph_loader.data.x, subgraph_loader.data.y
# Add global node index information.
subgraph_loader.data.num_nodes = data.num_nodes
subgraph_loader.data.n_id = torch.arange(data.num_nodes)

In [ ]:
train_loader = NeighborLoader(data, 
                            input_nodes=data.train_mask,
                            num_neighbors=[neighborhood_1, neighborhood_2], 
                            shuffle=True)

# TODO: not sure if subgraph is needed
subgraph_loader = NeighborLoader(copy.copy(data), input_nodes=None,
                                num_neighbors=[-1], shuffle=False)
# No need to maintain these features during evaluation:
del subgraph_loader.data.x, subgraph_loader.data.y
# Add global node index information.
subgraph_loader.data.num_nodes = data.num_nodes
subgraph_loader.data.n_id = torch.arange(data.num_nodes)